# Spam Filtering Using Gaussian Naive Bayes
---
Use the `spambase.data` file and the Gaussian Naive Bayes algorithm to build a simple spam detection algorithm. You can get accuracies around 80% with this simple classifier that is quickly trained.

In [355]:
import pandas as pd
import sklearn, sklearn.model_selection
import numpy as np

## Read the data

---

We need to read in the data from the spambase.data which is in CSV format. Pandas is perfect for this. If you look at the spambase.data file, you'll notice that there are no column headers and that the identification for spam vs. non-spam is found in column 57. So make sure you set the `header` argument to `None` and the `index_col` argument to `57`.

In [356]:
data = pd.read_csv('~/data/spam.data/spambase.data', header=None, index_col=57)

## Split the data

---

Now we want to split the data into training and testing sets. Scikit-learn has a great function for this: `sklearn.model_selection.train_test_split()`. The first parameter is the data to split, `data` in my case. We need to make sure to tell it to split the training and testing set in half by passing `0.5` to the `test_size` argument. Also, set the `random_state` argument to `np.random.RandomState()` so that the data is shuffled. (This is important since `spambase.data` is sorted by class).

In [357]:
X_train, X_test = sklearn.model_selection.train_test_split(data, test_size=0.5, random_state = np.random.RandomState())

## Get the prior probabilities

---

Note that the `spambase.data` file has approximately 40% of the emails categorized as spam, so our split should reflect that. Since the `train_test_split()` function returned our split training and testing sets as pandas dataframes and since we set the index column to be the class of the instance, when we say `X_train.loc[1]` we are selecting all of the rows in the training set that have class 1. That is, select all the spam instances. Similarly, we can select all the non-spam (or ham) instances by doing `X_train.loc[0]`. 

In [358]:
prob_spam_train = len(X_train.loc[1].index)/len(X_train.index)
prob_spam_train

0.4026086956521739

In [359]:
prob_ham_train = len(X_train.loc[0].index) / len(X_train.index)
prob_ham_train

0.5973913043478261

## Get the means and the standard deviations over the columns

---

In order to calculate the probability (which we simulate with the probability density function), we need to have the mean and standard deviation of each column for a given class (spam or ham). This is made quite easy by using the pandas [`describe()` function](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.describe.html). The result of `describe()` is a dataframe and to get the column means or standard deviations, we merely need to index the row labeled `mean` or `std` respectively.

In [360]:
train_spam_col_means = X_train.loc[1].describe().loc['mean']

In [361]:
train_spam_col_stds = X_train.loc[1].describe().loc['std']

In [362]:
train_ham_col_means = X_train.loc[0].describe().loc['std']

In [363]:
train_ham_col_stds = X_train.loc[0].describe().loc['std']

## Define a function to calculate the sum of the log of the probabilities

---

This function returns the result of the following equation:

$$\log{P(\textit{class})} + \sum_i \log{P(x_i \text{ | } \textit{class})}$$

Where 

$$P(x_i \text{ | } c_j) = N(x_i ; \mu_{i,c_j} , \sigma_{i, c_j})$$

and

$$N(x; \mu , \sigma) = \frac{1}{\sqrt{2\pi}\sigma}e^{-\frac{(x-\mu)^2}{2\sigma^2}}$$ 

Again, Pandas makes this easy. It'll be the case that `x`, `mus`, and `sigmas` are all pandas dataframes with 1 row and 57 columns, so we can add, subtract, multiply, eponentiate, log, or divide on an item-by-item basis. So `x - mus` is also a 1 x 57 dataframe, for example. Additionally, the pandas `sum()` function ignores `NaN` values by default, so there isn't anything we need to do to avoid them (i.e. when the std is 0).

In [364]:
def sum_of_log_probs(P_class, x, mus, sigmas):
    return np.log(P_class) + np.log((1 / (np.sqrt(2 * np.pi) * sigmas)) * np.exp(-((x - mus)**2 / (2 * sigmas**2)))).sum()

## Given an instance `x`, predict its class

---

Determine a class of an instance. Calculate the following

$$class_{NB}\big(\overrightarrow{\mathbf{x}}\big) = \underset{\textit{class} \in \{0, 1\}}{\mathrm{arg\,max}} \Big(\log{P(\textit{class})} + \sum_i \log{P(x_i \text{ | } \textit{class})} \Big)$$

where $P(x_i \text{ | } c_j)$ is as given previously.

In [365]:
def predict(x):
    pos = sum_of_log_probs(prob_spam_train, x, train_spam_col_means, train_spam_col_stds)
    neg = sum_of_log_probs(prob_ham_train, x, train_ham_col_means, train_ham_col_stds)
    return 1 if pos > neg else 0

## Classify all instances in the test set

---

Walk over the test set predicting each of the instances. Here's another time when pandas makes this nice. The `pandas.DataFrame.iterrows()` function returns a tuple `(index, row dataframe)`. So we have the actual value, which is stored in the index column since that's what was set by the `index_col` argument of the `pandas.read_csv()` function.

*Note: `numpy.log()` will return a runtime warning if the value it gets is equal to 0.0. It is fine for us to ignore this.*

In [366]:
cm = np.zeros((2,2))  # Define a blank confusion matrix
num_correct = 0
for actual, x in X_test.iterrows():
    predicted = predict(x)
    if actual == predicted:
        num_correct += 1
    cm[actual][predicted] += 1
print('accuracy: {} / {} = {:.4f}\n'.format(num_correct, len(X_test.index), num_correct / len(X_test.index)))
print(cm)

/usr/local/var/pyenv/versions/3.5.2/lib/python3.5/site-packages/ipykernel/__main__.py:2: RuntimeWarning: divide by zero encountered in log
  from ipykernel import kernelapp as app


accuracy: 1829 / 2301 = 0.7949

[[ 976.  438.]
 [  34.  853.]]
